# Python Crypto Notebook Worksheet #

The aim of this worksheet is to take you through some cryptography primitives using Python.

## Different crytographic terms

Cryptography is a generic term used to describe the design and anlysis of mechanisms based on mathematical techniques that provide fundamental security services. 

We can think of cryptography as the establishment of a large toolkit of different techniques, which can be used on their own or in combined, in security applications. 

A **cryptographic primitive ** is a cryptographic process that provides a number of specified security services. Cryptopgraphic primitives can be viewed as the basic generic tools in the cryptographic tool kit. E.g. block ciphers (deterministic algorithms operating on fixed-length groups of bits, called blocks, with an unvarying transformation specified by a symmetric key), message authentication codes, hash functions, and digital signature schemes. 

A **cryptographic algorithm** is the particular specification of a cryptographic primitive. E.g. AES is a cryptographic algorithm that specifies a block cipher. 

A **crypto system** is often used to refer to the implementation of some cryptographic primitives and their accompanying infrastructure. A cryptosystem used for confidentiality may consist of a block cipher as well as the users, the keys and the key management. 

## Cryptography concepts for developers
Source: an open source online book on practical cryptography https://cryptobook.nakov.com/

Cryptography is the science of providing security and protection of information. It is used everywhere in our digital world: when you open a Web site, send an email or connect to the WiFi network. That's why developers should have at least basic understanding of cryptography and how to use crypto algorithms and crypto libraries, to understand hashing, symmetric and asymmetric ciphers and encryption schemes, as well as digital signatures and the cryptosystems and algorithms behind them.

**Encryption and Keys**
Cryptography deals with storing and transmitting data in a secure way, such that only those, for whom it is intended, can read and process it. This may involve encrypting and decrypting data using symmetric or asymmetric encryption schemes, where one or more keys are used to transform data from plain to encrypted form and back. 

Symmetric encryption (like AES) uses the same key to encrypt and decrypt messages, while asymmetric encryption uses a public-key cryptosystem (like RSA) and a key-pair: public key (encryption key) and corresponding private key (decryption key).

Cryptography deals with **keys** (large secret numbers) and in many scenarios these keys are derived from numbers, passwords or passphrases using **key derivation algorithms** (like PBKDF2 and Scrypt).

**Digital Signatures** and **Message Authentication**
Cryptography provides means of digital signing of messages which guarantee message authenticity, integrity and non-repudiation. Most digital signature algorithms (like DSA, ECDSA and EdDSA) use asymmetric key pair (private and public key): the message is signed by the private key and the signature is verified by the corresponding public key. In the bank systems digital signatures are used to sign and approve payments. In blockchain signed transactions allow users to transfer a blockchain asset from one address to another.
Cryptography deals with message authentication algorithms (like HMAC) and message authentication codes (MAC codes) to prove message authenticity, integrity and authorship. Authentication is used side by side with encryption, to ensure secure communication.

**Secure Random Numbers**
Cryptography uses random numbers and deals with entropy (unpredictable randomness) and secure generation of random numbers (e.g. using CSPRNG). Secure random numbers are unpredictable by nature and developers should care about them, because broken random generator means compromised or hacked system or app.

**Key Exchange**
Cryptography defines key-exchange algorithms (like Diffie-Hellman key exchange) and key establishment schemes, used to securely establish encryption keys between two parties that intend to transmit messages securely using encryption. Such algorithms are performed typically when a new secure connection between two parties is established, e.g. when you open a modern Web site or connect to the WiFi network. 

**Cryptographic Hashes and Password Hashing**
Cryptography provides cryptographic hash functions (like SHA-3), which transform messages to message digest (hash of fixed length), which cannot be reversed back to the original message and almost uniquely identifies the input. In blockchain systems, for example, hashes are used to generate blockchain addresses, transaction ID and in many other algorithms and protocols. 

Password hashing and password to key derivation functions (like Scrypt) protect user passwords and password encrypted documents and data by securely deriving a hash (or key) from a text-based passwords, injecting random parameters (salt) and using a lot of iterations and computing resources to make password cracking slow.


In this python notebook, we are going to introduce some **common cryptographic primitives** that developers should know, and demonstrate some of their example use in python.

In [ ]:
# We mainly use the python crypto library pycrptodome 
# !pip install pycryptodomex 


# Encription and keys

First, we shall cover both symmetric-key and asymmetric-key (i.e. public-key) cryptography.
Symmetric ciphers use the same key to encrypt and decrypt, whereas asymmetric ciphers
have two keys. One of these is used to encrypt the data and the other is used to decrypt it.

Public-key cryptography uses this for authentication by a server keeping one key secret and
sharing the other key publicly. Depending on the key disclosed either the client
can encrypt a message with the public key that only the server may decrypt, or the client may
decrypt a message from the server which only the server could have encrypted.

Please read this overview for more information about symmetric and asymmetric cryptography
https://cryptobook.nakov.com/encryption-symmetric-and-asymmetric
or
http://www.garykessler.net/library/crypto.html#types 

In [ ]:
##
## This cell contains shared utility functions and variables
##

import Cryptodome as cd

# Base64 to make binary data easy to visually tell apart and copy/paste within
# the notebook
from base64 import b64encode, b64decode
# Asymmetric ciphers
from Cryptodome.Cipher import AES, DES3, PKCS1_OAEP
# Symmetric ciphers
from Cryptodome.PublicKey import RSA
# It is important to generate random bytes using a crypto library
# to ensure they have sufficient randomness (entropy). Whilst the operating system
# random number generation *should* be good enough, this cannot be
# guaranteed
from Cryptodome.Random import get_random_bytes

## Symmetric key cryptography
For this example, we shall use the [Advanced Encryption Standard (AES)](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) algorithm.


Again, more information on symmetric key cryptography can be found here http://www.garykessler.net/library/crypto.html#skc

In [ ]:
#  .+"+.+"+.+"+.+"+.+"+.+"+.
# (                         )
#  ) Symmetric key example (
# (                         )
#  "+.+"+.+"+.+"+.+"+.+"+.+"


# The data to encrypt. Note that the string is prepended with a 'b'
# to tell python to coerce it to a byte array, as encryption can
# only be performed on binary data
data = b"Here is some test data"

print('Input data:\n\t{}\n\t{}'.format(data, b64encode(data)))

##########################
## Setup and encryption ##
##########################

# To get a 128 bit key (1 byte=8 bit)
aes_key = get_random_bytes(16)

# A mode of operation (https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation) 
# describes how to repeatedly apply a cipher's single-block
# operation to securely transform any amounts of data larger than a block.
# AES mode is used here to simultaneously provide both authentication and privacy 
# of the message (authenticated encryption) with a two-pass scheme. 
aes_cipher = AES.new(aes_key, AES.MODE_EAX)
aes_nonce = aes_cipher.nonce 
aes_ciphertext, aes_tag = aes_cipher.encrypt_and_digest(data)

print('\nKey:\t{}\nNonce:\t{}\nTag:\t{}\n'.format(b64encode(aes_key), # key should be kept secret
                                              b64encode(aes_nonce),   # initialisation vector
                                              b64encode(aes_tag)))    # used for authentication here

print('Encrypted data:\n\t{}\n\t{}'.format(aes_ciphertext, b64encode(aes_ciphertext)))


file_out = open("encrypted.bin", "wb")
[ file_out.write(x) for x in (aes_nonce, aes_tag, aes_ciphertext)]
file_out.close()
 
################
## Decryption ##
################
# The receiver can securely load the piece of data back (if they know the key!).
file_in = open("encrypted.bin", "rb")
aes_nonce1, aes_tag1, aes_ciphertext1 = [file_in.read(x) for x in (16, 16, -1) ]
file_in.close()
print('\n\nLoading saved configurations: \nCiphertext:\t{}\nNonce:\t{}\nTag:\t{}\n'.format(b64encode(aes_ciphertext1), # key should be kept secret
                                              b64encode(aes_nonce1),   # initialisation vector
                                              b64encode(aes_tag1)))    # used for authentication here


# Create an AES cipher using the key we encryped with and nonce that was generated
decrypt_cipher = AES.new(aes_key, AES.MODE_EAX, aes_nonce1)
decrypted_data = decrypt_cipher.decrypt_and_verify(aes_ciphertext1, aes_tag1)


print('Decrypted data:\n\t{}\n\t{}'.format(decrypted_data, b64encode(decrypted_data)))

In [ ]:
# Now given the following encrypted data, key, nonce, and tag,
# decrypt it to find the message
aes_test_key = base64.b64decode('AWbhqolEPuizxqVE12q/PA==')
aes_test_nonce = base64.b64decode('Qj/ck4qYarb1z9/5WqafKw==')
aes_test_tag = base64.b64decode('uMjq5w2Ol8ERKSxJYwHnCA==')
aes_test_ciphertext = base64.b64decode('4nqdyUbMcmtFYeQ3ihKSJRAB8YnfL8SDLDSW7gbPVdyve5zr8E71OQJ3Lw==')

# Fix me!
decrypted_data = '?'
print('Message: {}\n'.format(decrypted_data))

## Asymmetric key cryptography

Asymmetric key cryptography (public-key cryptography) is vital for establishing trust between two parties.

Please see [Gary Kessler's Web of Trust section](http://www.garykessler.net/library/crypto.html#trust)

This example shall use the [Rivest–Shamir–Adleman (RSA)](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) algorithm.

More information on symmetric key cryptography is available at http://www.garykessler.net/library/crypto.html#pkc

In [ ]:
#  .+"+.+"+.+"+.+"+.+"+.+"+.
# (                         )
#  )Asymmetric key example (
# (                         )
#  "+.+"+.+"+.+"+.+"+.+"+.+"


# The data to encrypt
data = b'Some more test data'

print('Input data:\n\t{}\n\t{}\n'.format(data, b64encode(data)))

# Generate a random RSA key
rsa_key = RSA.generate(2048)

rsa_cipher = PKCS1_OAEP.new(rsa_key)
rsa_ciphertext = rsa_cipher.encrypt(data)
print('Encrypted message:\n\t{}\n'.format(b64encode(rsa_ciphertext)))

rsa_decrypted_message = rsa_cipher.decrypt(rsa_ciphertext)
print('Decrypted message:\n\t{}\n\t{}\n'.format(rsa_decrypted_message, b64encode(rsa_decrypted_message)))

print('Public key:\n{}\n'.format(rsa_key.publickey().exportKey().decode('utf-8')))
print('Private key:\n{}\n'.format(rsa_key.exportKey().decode('utf-8')))

In [ ]:
# Now, like in the last example, given a key
# you must decrypt the message

rsa_test_ciphertext = b64decode('J/Xvl57JJu4Z/7NVVvCtibJy3ne6aWlafDtKfTYVUMi8poY1WBPLdizvIsI1cfvyrbRfNmVrJgcy69byxrka6EvD7Zq32nHwhhJG1rvT1UGCDTvsmx7j/E1Ayh26uUWNsSFkuztlB5bTLCHZgqXMNYzY5YFoiqeqeZe7wuuplqA2hRM3ztNpOY6hYf6qvPwd04IIuBX8+EKwJzNv21SYVsbSYJ8vn6ozw6tXyTHcWBiECJIfr6+DfQNNf7fmlSGXPjPFHdZc8lB59TurZX0+MDsChZmMKuqKsJa4i+cX3likknmr/jSC3M5SzVqJyB/4jEusRbYlj3cBDuqxaLgW9Q==')
rsa_test_pubkey = RSA.import_key(b64decode('MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAxH5udlRxQryLY2pGAi62hkZBTKcTb5u4M05+3aSKlaX2DKuDzJad/ryYgkL2d5dwKqkc+JkrE1GW4iSpmxOEZ5ngVJ2uPuYp/YtaIr+032fYj25hl4qwodJBqVHu2S2vVQLV1q54B0lXCLJYfZfbnelxb8EFdHNPKvjlXV1P0wXb7mdUk5ALvUI7jF/WNj2I99WTGjtRpQ2gUDmf1NQ0EAOoQQKCMw2MqSPLCCSeqS/DXhhxeonszt6e8YQaScqQH7epsFF6ktCScU+l6nWdQ1bw0/itxmLG5zcbUKEuYEnWPXaHDSaChIphIqzEHIy3d2OGydQOwMrinEBQ12I7swIDAQAB'))

# Fix me too!
rsa_decrypted_message = '?'
print('Decrypted message: {}'.format(rsa_decrypted_message))

# Hash function
Cryptographic hash functions transform text or binary data to fixed-length hash value and are known to be collision-resistant and irreversible. Collision happens when the different input texts have got the same hash value. Example of cryptographic hash function is SHA3-256. 

In [ ]:
from Cryptodome.Hash import SHA256, SHA512


#A first piece of message to hash can be passed to new() with the data parameter
hash_object = SHA256.new(data=b'First')

# update invoked any number of times as necessary, with other pieces of message:
hash_object.update(b'Second') # 
hash_object.update(b'Third')

#At the end, the digest (or hash value) can be retrieved with the methods digest() or hexdigest()
print(hash_object.digest())
print(hash_object.hexdigest())


Questions: 
- What are the other Hash algorithms besides SHA256? 
- Are they secured?


# Digital Signatures and Message Authentication

Digital signatures are a cryptographic tool to sign messages and verify message signatures in order to provide proof of authenticity for digital messages or electronic documents. Digital signatures provide:
Message authentication - a proof that certain known sender (secret key owner) have created and signed the message.
Мessage integrity - a proof that the message was not altered after the signing.
Non-repudiation - the signer cannot deny the signing of the document after the signature is once created.

For more information go to:
https://cryptobook.nakov.com/digital-signatures


In [ ]:
keyPair = RSA.generate(bits=1024)
print(f"Public key:  (n={hex(keyPair.n)}, e={hex(keyPair.e)})")
print(f"Private key: (n={hex(keyPair.n)}, d={hex(keyPair.d)})")


In [ ]:
# Now, let's sign a message, using the RSA private key {n, d}. 
# Calculate its hash and raise the hash to the power d modulo n 
#(encrypt the hash by the private key). In Python we have modular exponentiation 
# as built in function pow(x, y, n):

# RSA sign the message
msg = b'A message for signing'
from hashlib import sha512
print(sha512(msg).digest())
hash = int.from_bytes(SHA512.new(data=msg).digest(), byteorder='big')
signature = pow(hash, keyPair.d, keyPair.n)

# The obtained digital signature is an integer in the range of the RSA key length [0...n). 
print("Signature:", hex(signature)) 

In [ ]:
# The signature is 1024-bit integer (128 bytes, 256 hex digits). This signature size corresponds to the RSA key size.
# Now, let's verify the signature, by decrypting the signature using the public key 
# (raise the signature to power e modulo n) and comparing the obtained hash from the signature to the hash of the 
# originally signed message

# RSA verify signature
msg = b'A message for signing'
#hash = int.from_bytes(sha512(msg).digest(), byteorder='big')
hash = int.from_bytes(SHA512.new(data=msg).digest(), byteorder='big')

hashFromSignature = pow(signature, keyPair.e, keyPair.n)
print("Signature valid:", hash == hashFromSignature)

# Check if the singuature is still valid if the key is changed. 
# ---

## Final notes

Cryptography is a complex field. While it's easy to use cryptography (as shown above), it is often difficult to
do it right. As an example from this worksheet, the RSA key generated is not encrypted and thus would be trivial to steal. For the sake of simplicity this was left out, but there are many extra considerations over data storage and transfer when dealing with encrypted data.

For a very thorough look at cryptography's history and current role in computing, see the [infosec institute's crypto post](http://resources.infosecinstitute.com/role-of-cryptography/)

Be sure to read up on good practices at sites such as [OWASP](https://www.owasp.org/index.php/Main_Page)

OWASP provides a [pdf on secure coding practices](https://www.owasp.org/images/0/08/OWASP_SCP_Quick_Reference_Guide_v2.pdf) as well as a [cheatsheet reference](https://github.com/OWASP/CheatSheetSeries)

Another useful resource on cryptography is the [Awesome Cryptography List](https://github.com/sobolevn/awesome-cryptography), which provides links to learning resouces covering general cryptography subjects as well as specific frameworks for a variety of languages.